we'll put everything together and create our model

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import pickle
import cv2
%matplotlib inline

<b>loading all models

In [5]:
haar=cv2.CascadeClassifier('./model/haarcascade_frontalface_default.xml')
mean=pickle.load(open('./model/mean_preprocessing.pickle', 'rb'))
model_svm=pickle.load(open('./model/model_svm.pickle', 'rb'))
model_pca=pickle.load(open('./model/pca_50.pickle', 'rb'))
print('models loaded successfully')

models loaded successfully


<b>our test data</b>

In [31]:
test_data_path='./data/male_000281.jpg'

steps-
1. read image
2. convert into grayscale
3. crop face using haar acscade classifier
4. normalize the image
5. resize image to 100x100
6. flattening the image (reshape to 1x10,000)
7. subtract with mean
8. get eigen image
9. pass to ML model (SVM)
10. picking highest probability
11. generate text

In [32]:
gender_pre=['Male','Female']
font=cv2.FONT_HERSHEY_SIMPLEX

In [10]:
def pipeline_model(img,color='rgb'):
    #s2-
    if color =='bgr':
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #s3-
    faces=haar.detectMultiScale(gray,1.5,3)
    for x,y,w,h in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2) #drawing rectangle
        roi=gray[y:y+h,x:x+w] #cropping image
        #s4- normalization
        roi=roi/255.0
        #s5-
        if roi.shape[1] > 100:
            roi_resize=cv2.resize(roi,(100,100),cv2.INTER_AREA)
        else:
            roi_resize=cv2.resize(roi,(100,100),cv2.INTER_CUBIC)
        #s6-
        roi_reshape=roi_resize.reshape(1,10000) # or 1,-1
        #s7- 
        roi_mean=roi_reshape-mean
        #s8-
        eigen_image=model_pca.transform(roi_mean)
        #s9-
        results=model_svm.predict_proba(eigen_image)[0]
        #s10-picking highest probability
        predict=results.argmax() # 0 or 1
        score=results[predict]
        #s11- 
        text="%s : %0.2f"%(gender_pre[predict], score)
        cv2.putText(img, text, (x,y), font, 1, (0,255,0), 2)
    return img
    

for deploying purpose, since we are reading an image in bgr. if we save our image in bgr, the html may not take bgr format.

Steps-
1. read an image using PIL
2. convert into array using numpy
3. pass to pipeline model